In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from datetime import *
import matplotlib.pylab as pylab
from pylab import *  
mpl.rcParams['font.sans-serif'] = ['SimHei'] 


In [2]:
#读取前期处理过后的训练集和测试集进行窗口数据集划分
train_df = pd.read_csv('chusai_train_df.csv')
test_df = pd.read_csv('chusai_test_df.csv')

print(train_df.columns.values)
print(test_df.columns.values)


['instance_id' 'item_id' 'item_category_list' 'item_property_list'
 'item_brand_id' 'item_city_id' 'item_price_level' 'item_sales_level'
 'item_collected_level' 'item_pv_level' 'user_id' 'user_gender_id'
 'user_age_level' 'user_occupation_id' 'user_star_level' 'context_id'
 'context_timestamp' 'context_page_id' 'predict_category_property'
 'shop_id' 'shop_review_num_level' 'shop_review_positive_rate'
 'shop_star_level' 'shop_score_service' 'shop_score_delivery'
 'shop_score_description' 'is_trade' 'date' 'weekday' 'day' 'hour'
 'context_page_id_1' 'context_page_id_2' 'context_page_id_3'
 'context_page_id_4' 'item_price_level_0' 'item_price_level_1'
 'item_pv_level_low' 'shop_review_num_level_lowHigh'
 'shop_star_level_lowHigh' 'shop_star_level_middle' 'shop_star_level_loss'
 'day_hour' 'predict_category_list' 'predict_category_set'
 'real_item_category_list' 'predict_property_list'
 'match_category_proportion' 'match_property_proportion'
 'predict_category_number' 'predict_property_num

In [3]:
#开始进行训练集窗口数据集的划分
#18，19，20---21
#19，20，21---22
#20，21，22---23
#21，22，23---24
future_dataset1 = train_df[(train_df.day >= 18) & (train_df.day <= 20)]
train_dataset1 = train_df[train_df.day == 21]
future_dataset2 = train_df[(train_df.day >= 19) & (train_df.day <= 21)]
train_dataset2 = train_df[train_df.day == 22]
future_dataset3 = train_df[(train_df.day >= 20) & (train_df.day <= 22)]
train_dataset3 = train_df[train_df.day == 23]
future_dataset4 = train_df[(train_df.day >= 21) & (train_df.day <= 23)]
train_dataset4 = train_df[train_df.day == 24]

#开始进行测试集窗口数据集的划分
#22，23，24---25
future_dataset5 = train_df[(train_df.day >= 22) & (train_df.day <= 24)]
test_dataset = test_df.copy()


In [4]:
#统计三天前单特征的转化率
def getBayesSmoothParam(origion_rate):
    origion_rate_mean = origion_rate.mean()
    origion_rate_var = origion_rate.var()
    alpha = origion_rate_mean / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
    beta = (1 - origion_rate_mean) / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
#     print('origion_rate_mean : ', origion_rate_mean)
#     print('origion_rate_var : ', origion_rate_var)
#     print('alpha : ', alpha)
#     print('beta : ', beta)
    return alpha, beta

#为用户，商品，店铺，城市，叶子类目，根类目定义基础转化率，点击次数，购买次数特征
def getBaseConversionRate(future_df, test_df, colName):
    t = future_df[[colName]]
    t[colName + '_total_number'] = 1
    t = t.groupby(colName).agg('sum').reset_index()

    t_buy = future_df[[colName]][future_df.is_trade == 1]
    t_buy[colName + '_buy_number'] = 1
    t_buy = t_buy.groupby(colName).agg('sum').reset_index()

    t = pd.merge(t, t_buy, on=colName, how='left')
    t[colName + '_buy_number'] = t[colName + '_buy_number'].map(lambda x: 0 if math.isnan(x) else x)
    t['buy_origion_rate'] = t[colName + '_buy_number'] / t[colName + '_total_number']
    alpha, beta = getBayesSmoothParam(t['buy_origion_rate'])
    t[colName + '_converse_smooth_rate'] = (t[colName + '_buy_number'] + alpha) / (t[colName + '_total_number'] + alpha + beta)
#     train_df = pd.merge(train_df, t[[colName, colName + '_converse_smooth_rate']], on=colName, how='left')
#     train_df[colName + '_converse_smooth_rate'] = train_df[colName + '_converse_smooth_rate'].map(lambda x: (alpha / (alpha + beta)) if math.isnan(x) else x)
    
    test_df = pd.merge(test_df, t[[colName, colName + '_converse_smooth_rate', colName + '_total_number', colName + '_buy_number']], on=colName, how='left')
    test_df[colName + '_converse_smooth_rate'] = test_df[colName + '_converse_smooth_rate'].map(lambda x: (alpha / (alpha + beta)) if math.isnan(x) else x)
    
    test_df[colName + '_total_number'] = test_df[colName + '_total_number'].map(lambda x: 0 if math.isnan(x) else x)
    
    test_df[colName + '_buy_number'] = test_df[colName + '_buy_number'].map(lambda x: 0 if math.isnan(x) else x)
    
    return test_df

#定义对每个窗口进行操作的函数
def dealHuaChuangDataset(future_df, dataset):
    dataset = getBaseConversionRate(future_df, dataset, 'user_id')
    dataset = getBaseConversionRate(future_df, dataset, 'item_brand_id')
    dataset = getBaseConversionRate(future_df, dataset, 'item_id')
    dataset = getBaseConversionRate(future_df, dataset, 'shop_id')
    dataset = getBaseConversionRate(future_df, dataset, 'item_city_id')
    dataset = getBaseConversionRate(future_df, dataset, 'real_first_category')
    dataset = getBaseConversionRate(future_df, dataset, 'real_last_category')
    dataset = getBaseConversionRate(future_df, dataset, 'hour')
    return dataset

print(len(train_dataset1.columns))
train_dataset1 = dealHuaChuangDataset(future_dataset1, train_dataset1)
train_dataset2 = dealHuaChuangDataset(future_dataset2, train_dataset2)
train_dataset3 = dealHuaChuangDataset(future_dataset3, train_dataset3)
train_dataset4 = dealHuaChuangDataset(future_dataset4, train_dataset4)
test_dataset = dealHuaChuangDataset(future_dataset5, test_dataset)
print(len(train_dataset1.columns))


117


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


141


In [5]:
#为组合特征定义点击量，购买量，转化率特征
def getZuheFutureConversionRate(future_df, test_df, colName1, colName2, newColName):
    t_train = future_df[[colName1, colName2, 'is_trade']]
    t_train[colName1 + '_str'] = t_train[colName1].map(lambda x: str(x))
    t_train[colName2 + '_str'] = t_train[colName2].map(lambda x: str(x))
    t_train[newColName] = t_train[colName1 + '_str'] + t_train[colName2 + '_str']
    
    t_test = test_df[[colName1, colName2]]
    t_test[colName1 + '_str'] = t_test[colName1].map(lambda x: str(x))
    t_test[colName2 + '_str'] = t_test[colName2].map(lambda x: str(x))
    t_test[newColName] = t_test[colName1 + '_str'] + t_test[colName2 + '_str']
    
    t_test = getBaseConversionRate(t_train, t_test, newColName)
    test_df[newColName + '_converse_smooth_rate'] = t_test[newColName + '_converse_smooth_rate']
    test_df[newColName + '_total_number'] = t_test[newColName + '_total_number']
    test_df[newColName + '_buy_number'] = t_test[newColName + '_buy_number']
    
    return test_df

#初步定义跟用户相关的组合特征点击量，购买量，转化率
def dealUserIdZuheRateHuaChuangDataset(future_df, dataset):
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_id', 'item_id', 'user_item')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_id', 'item_brand_id', 'user_brand')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_id', 'shop_id', 'user_shop')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_id', 'item_city_id', 'user_city')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_id', 'real_first_category', 'user_first_category')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_id', 'real_last_category', 'user_last_category')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_id', 'item_price_level', 'user_item_price')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_id', 'item_sales_level', 'user_item_sales')
    return dataset

print(len(train_dataset1.columns))
train_dataset1 = dealUserIdZuheRateHuaChuangDataset(future_dataset1, train_dataset1)
train_dataset2 = dealUserIdZuheRateHuaChuangDataset(future_dataset2, train_dataset2)
train_dataset3 = dealUserIdZuheRateHuaChuangDataset(future_dataset3, train_dataset3)
train_dataset4 = dealUserIdZuheRateHuaChuangDataset(future_dataset4, train_dataset4)
test_dataset = dealUserIdZuheRateHuaChuangDataset(future_dataset5, test_dataset)
print(len(train_dataset1.columns))


141


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

165


In [6]:
#初步定义跟用户用户性质和商品品牌，店铺组合特征点击量，购买量，转化率
def dealUserZuheRateHuaChuangDataset(future_df, dataset):
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_gender_id', 'item_id', 'user_gender_item')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_gender_id', 'item_brand_id', 'user_gender_brand')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_gender_id', 'shop_id', 'user_gender_shop')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_age_level', 'item_id', 'user_age_item')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_age_level', 'item_brand_id', 'user_age_brand')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_age_level', 'shop_id', 'user_age_shop')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_occupation_id', 'item_id', 'user_occupation_item')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_occupation_id', 'item_brand_id', 'user_occupation_brand')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_occupation_id', 'shop_id', 'user_occupation_shop')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_star_level', 'item_id', 'user_star_item')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_star_level', 'item_brand_id', 'user_star_brand')
    dataset = getZuheFutureConversionRate(future_df, dataset, 'user_star_level', 'shop_id', 'user_star_shop')
    return dataset

print(len(train_dataset1.columns))
train_dataset1 = dealUserZuheRateHuaChuangDataset(future_dataset1, train_dataset1)
train_dataset2 = dealUserZuheRateHuaChuangDataset(future_dataset2, train_dataset2)
train_dataset3 = dealUserZuheRateHuaChuangDataset(future_dataset3, train_dataset3)
train_dataset4 = dealUserZuheRateHuaChuangDataset(future_dataset4, train_dataset4)
test_dataset = dealUserZuheRateHuaChuangDataset(future_dataset5, test_dataset)
print(len(train_dataset1.columns))


165


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

201


In [7]:
#将滑窗训练集进行合并导出
huachuang_train_df = pd.concat([train_dataset1, train_dataset2, train_dataset3, train_dataset4])
huachuang_test_df = test_dataset

# 导出滑窗训练集预处理结果
def exportResult(df, fileName):
    df.to_csv('./%s.csv' % fileName, header=True, index=False)
    
exportResult(huachuang_train_df, 'chusai_huachuang_train_df')
exportResult(huachuang_test_df, 'chusai_huachuang_test_df')


In [8]:
print(huachuang_train_df.columns.values)

['instance_id' 'item_id' 'item_category_list' 'item_property_list'
 'item_brand_id' 'item_city_id' 'item_price_level' 'item_sales_level'
 'item_collected_level' 'item_pv_level' 'user_id' 'user_gender_id'
 'user_age_level' 'user_occupation_id' 'user_star_level' 'context_id'
 'context_timestamp' 'context_page_id' 'predict_category_property'
 'shop_id' 'shop_review_num_level' 'shop_review_positive_rate'
 'shop_star_level' 'shop_score_service' 'shop_score_delivery'
 'shop_score_description' 'is_trade' 'date' 'weekday' 'day' 'hour'
 'context_page_id_1' 'context_page_id_2' 'context_page_id_3'
 'context_page_id_4' 'item_price_level_0' 'item_price_level_1'
 'item_pv_level_low' 'shop_review_num_level_lowHigh'
 'shop_star_level_lowHigh' 'shop_star_level_middle' 'shop_star_level_loss'
 'day_hour' 'predict_category_list' 'predict_category_set'
 'real_item_category_list' 'predict_property_list'
 'match_category_proportion' 'match_property_proportion'
 'predict_category_number' 'predict_property_num